In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import math
import numpy as np
%matplotlib inline

In [18]:
# data_path = r'C:\Users\johnn\Downloads\nba_odds_raw.csv'
data_path = r'C:\Users\johnny.wong\PycharmProjects\NBA pricing\Data\nba_odds_raw.csv'
raw_odds = pd.read_csv(data_path)
raw_odds.head()

,YY,DD,MM,Seasons,Home team,Away team,Home ID,Away ID,End of regulation score,Home final score,Away final score,Winning team,Total score,Home odds 1,Away odds 1,Home odds 2,Away odds 2,Home odds 3,Away odds 3
0,2019,11,Feb,2018/2019,Golden State Warriors,Miami Heat,GSW,MIA,NaN,120.0,118.0,H,238.0,1.08,10.40,1.08,10.16,1.07,9.50
1,2019,11,Feb,2018/2019,Atlanta Hawks,Orlando Magic,ATL,ORL,NaN,108.0,124.0,A,232.0,2.06,1.62,2.04,1.87,2.20,1.71
2,2019,10,Feb,2018/2019,Sacramento Kings,Phoenix Suns,SAC,PHX,NaN,117.0,104.0,H,221.0,1.19,3.90,1.24,4.65,1.26,4.00
3,2019,10,Feb,2018/2019,Philadelphia 76ers,Los Angeles Lakers,PHI,LAL,NaN,143.0,120.0,H,263.0,1.33,3.00,1.34,3.56,1.34,3.35
4,2019,10,Feb,2018/2019,Dallas Mavericks,Portland Trail Blazers,DAL,POR,NaN,102.0,101.0,H,203.0,2.61,1.57,2.62,1.56,1.74,2.15


In [19]:
clean_data = raw_odds.copy()
clean_data['date_string'] = clean_data.apply(lambda row:str(row.DD)+'-'+str(row.MM)+'-'+str(row.YY), axis=1)
clean_data['Date'] = clean_data.date_string.apply(lambda date_str: dt.datetime.strptime(date_str, '%d-%b-%Y'))

# Remove bad data
clean_data = clean_data[raw_odds['Total score'] > 50]

# Used for later since there seems to be positive skew
clean_data['log Total score'] = clean_data['Total score'].apply(math.log)
clean_data.head()

,YY,DD,MM,Seasons,Home team,Away team,Home ID,Away ID,End of regulation score,Home final score,...,Total score,Home odds 1,Away odds 1,Home odds 2,Away odds 2,Home odds 3,Away odds 3,date_string,Date,log Total score
0,2019,11,Feb,2018/2019,Golden State Warriors,Miami Heat,GSW,MIA,NaN,120.0,...,238.0,1.08,10.40,1.08,10.16,1.07,9.50,11-Feb-2019,2019-02-11,5.472271
1,2019,11,Feb,2018/2019,Atlanta Hawks,Orlando Magic,ATL,ORL,NaN,108.0,...,232.0,2.06,1.62,2.04,1.87,2.20,1.71,11-Feb-2019,2019-02-11,5.446737
2,2019,10,Feb,2018/2019,Sacramento Kings,Phoenix Suns,SAC,PHX,NaN,117.0,...,221.0,1.19,3.90,1.24,4.65,1.26,4.00,10-Feb-2019,2019-02-10,5.398163
3,2019,10,Feb,2018/2019,Philadelphia 76ers,Los Angeles Lakers,PHI,LAL,NaN,143.0,...,263.0,1.33,3.00,1.34,3.56,1.34,3.35,10-Feb-2019,2019-02-10,5.572154
4,2019,10,Feb,2018/2019,Dallas Mavericks,Portland Trail Blazers,DAL,POR,NaN,102.0,...,203.0,2.61,1.57,2.62,1.56,1.74,2.15,10-Feb-2019,2019-02-10,5.313206


In [31]:
team_list = ["ATL","BKN","BOS","CHA","CHI","CLE","DAL","DEN","DET","GSW","HOU","IND",
             "LAC","LAL","MEM","MIA","MIL","MIN","NOP","NYK","OKC","ORL","PHI","PHX",
             "POR","SAC","SAS","TOR","UTA","WAS"]

def filter_by_team(df, team):
    df_team = df[(df['Home ID'] == team)|(df['Away ID'] == team)]
    return df_team

def emp_option_theo(option, strike, df, shift=0):
    '''
    Returns the theo and probability of ending above the strike based on empirical data provided in df
    '''
    if option not in ['C','P']:
        raise ValueError('option must be "C" or "P"')
    n = float(len(df))

    if option == 'C':
        total_payoff = (df['Total score'] + shift).apply(lambda score: 0 if score < strike else score - strike).sum()
    elif option == 'P':
        total_payoff = (df['Total score'] + shift).apply(lambda score: 0 if score > strike else strike - score).sum()

    value = total_payoff/n
    
    # Probability of call ITM
    call_delta = df['Total score'] > strike).sum()/n
    return value#, call_delta

In [32]:
df_this_season = clean_data[clean_data['Seasons'] == '2018/2019']

In [38]:

df_theos = pd.DataFrame(data={
    'strike':list(range(150, 360, 10))
})

shift = 11
df_theos['Call theo'] = df_theos.strike.apply(lambda K: emp_option_theo('C', K, df_this_season, shift))
df_theos['Put theo'] = df_theos.strike.apply(lambda K: emp_option_theo('P', K, df_this_season, shift))
df_theos['Implied underlying'] = df_theos['Call theo'] - df_theos['Put theo'] + df_theos['strike']
df_theos

,strike,Call theo,Put theo,Implied underlying
0,150,81.917559,0.000000,231.917559
1,160,71.917559,0.000000,231.917559
2,170,61.917559,0.000000,231.917559
3,180,51.917559,0.000000,231.917559
4,190,41.947537,0.029979,231.917559
5,200,32.243041,0.325482,231.917559
6,210,23.222698,1.305139,231.917559
7,220,15.297645,3.380086,231.917559
8,230,9.143469,7.225910,231.917559
9,240,4.791221,12.873662,231.917559
